# Crash Data Big Data Processing

In [10]:
# Package Upload

import pandas as pd
from pathlib import Path

# Data Upload

## Three files (NOTE THAT THEY DO NOT USE UTF-8 ENCODING! ALSO: THE FIRST ROW IS JUST A HYPERLINK!)
df_0 = pd.read_csv("../data/CT_crashes/export_5673_0.csv", encoding="cp1252", skiprows=1)
df_1 = pd.read_csv("../data/CT_crashes/export_5673_1.csv", encoding="cp1252", skiprows=1)
df_2 = pd.read_csv("../data/CT_crashes/export_5673_2.csv", encoding="cp1252", skiprows=1)

/var/folders/lt/90cmmvp91vn9qgdwpxyyb6l40000gn/T/ipykernel_81449/3423538044.py:9: DtypeWarning: Columns (32,84,100,102) have mixed types. Specify dtype option on import or set low_memory=False.
  df_0 = pd.read_csv("../data/CT_crashes/export_5673_0.csv", encoding="cp1252", skiprows=1)
/var/folders/lt/90cmmvp91vn9qgdwpxyyb6l40000gn/T/ipykernel_81449/3423538044.py:10: DtypeWarning: Columns (7,76,81,88,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1 = pd.read_csv("../data/CT_crashes/export_5673_1.csv", encoding="cp1252", skiprows=1)


## Individual File Processing

In [3]:
# Keep Columns Lists:

## dataset 0:

keep_columns_0 = [
        "CrashId",
        "Fatal Case Status",
        "Latitude",
        "Longitude",
        "Date of Crash",
        "Day of the Week",
        "Day of the Week Text Format",
        "Hour of the Day",
        "Most Severe Injury",
        "Number Of Motor Vehicles",
        "Number Of Non-Motorist",
        "Route Class", "Route Class Text Format",
        "First Harmful Event",
        "Manner of Crash / Collision Impact",
        "Location of First Harmful Event",
        "Weather Condition", "Weather Condition Text Format",
        "Light Condition", "Light Condition Text Format",
        "Road Surface Condition", "Road Surface Condition Text Format",
        "Crash Specific Location", "Crash Specific Location Text Format",
        "Type of Intersection", "Type of Intersection Text Format"
    ]

## dataset 1:

keep_columns_1 = [
        "CrashId", "VehicleId", "Vehicle Unit Type", "Make", 
        "Vehicle Model Year", "Model", "Vehicle Color", 
        "Most Harmful Event", "Most Harmful Event Text Format",
        "Vehicle Manuever/Action", "Vehicle Manuever/Action Text Format",
        "Roadway Grade", "Extent of Damage", "Extent of Damage Text Format",
        "Body Type", "Body Type Text Format"
    ]

## dataset 2:

keep_columns_2 = [
        "CrashId", "Age", "Gender", "Gender Text Format", "State", "Postal Code",
        "Person Type", "Person Type Text Format", "Injury Status",
        "Air Bag Status", "Air Bag Status Text Format",
        "Speeding Related", "Speeding Related Text Format"
    ]


In [4]:
# Precursor functions for Data Cleaning:

## IDENTIFY NUMERIC, TEXT FIELDS TO EXPORT TEXT VERSION TO DECODING FILE FOR STORAGE PRESERVATION
def extract_decodings(df, decoding_records):
    """
    Extract unique code → text mappings from columns ending in ' Text Format'
    and append them to decoding_records (a list of dicts).
    """
    text_cols = [c for c in df.columns if c.endswith(" Text Format")]

    for text_col in text_cols:
        code_col = text_col.replace(" Text Format", "")

        if code_col not in df.columns:
            continue

        pairs = (
            df[[code_col, text_col]]
            .dropna()
            .drop_duplicates()
        )

        for _, row in pairs.iterrows():
            decoding_records.append({
                "variable": code_col,
                "code": row[code_col],
                "text": row[text_col]
            })

    # Drop text columns after extraction
    df = df.drop(columns=text_cols)

    return df

## TAKE AN INITIAL DATAFRAME (0,1,2) AND USE ITS KEEP_COLUMNS TO REMOVE UNNECESSARY INFO
def reduce_single_df(df, keep_columns, decoding_records):
    """
    Reduce a single dataframe:
    - Keep only desired columns
    - Extract decoding info
    - Remove 'Text Format' columns
    """

    # Keep only requested columns (safe intersection)
    df = df.loc[:, df.columns.intersection(keep_columns)]

    # Extract and remove text-format columns
    df = extract_decodings(df, decoding_records)

    return df

In [5]:
# DATA CLEANING EXECUTION FUNCTION
def clean_and_merge_crash_data(
    dfs,
    keep_columns_list,
    output_dir="../data/clean_crash_data"
):
    """
    Parameters
    ----------
    dfs : list[pd.DataFrame]
        List of crash dataframes
    keep_columns_list : list[list[str]]
        Matching list of keep-columns for each df
    """

    Path(output_dir).mkdir(parents=True, exist_ok=True)

    decoding_records = []
    cleaned_dfs = []

    for df, keep_cols in zip(dfs, keep_columns_list):
        reduced = reduce_single_df(df, keep_cols, decoding_records)
        cleaned_dfs.append(reduced)

    # ---- Merge datasets ----
    merged = pd.concat(cleaned_dfs, axis=0, ignore_index=True)

    # ---- Remove duplicate columns by value ----
    merged = merged.T.drop_duplicates().T

    # ---- Save clean crash data ----
    clean_path = Path(output_dir) / "clean_crash_data.csv"
    merged.to_csv(clean_path, index=False)

    # ---- Build and save decoding table ----
    if decoding_records:
        decoding_df = (
            pd.DataFrame(decoding_records)
            .drop_duplicates()
            .sort_values(["variable", "code"])
        )
    else:
        decoding_df = pd.DataFrame(columns=["variable", "code", "text"])

    decoding_path = Path(output_dir) / "decoding_table.csv"
    decoding_df.to_csv(decoding_path, index=False)

    return merged, decoding_df

## Merging Files Together

### Reducing file size significantly

In [ ]:
merged_df, decoding_df = clean_and_merge_crash_data(
    dfs=[df_0, df_1, df_2],
    keep_columns_list=[keep_columns_0, keep_columns_1, keep_columns_2]
)

In [7]:
print(merged_df)

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[1612723 rows x 0 columns]


## Building Decoding Files

In [ ]:
# First Harmful Event
# Most severe injury
# Manner of Crash / Collision Impact
# Location of First Harmful Evenet